In [ ]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
import pandas as pd

archivo = 'zara.csv'
df = pd.read_csv(archivo)
df.head()

# Agente Taylor(ed)

In [ ]:
llm = ChatOpenAI(temperature=0.5,
                 model_name = 'gpt-4o-mini')

In [ ]:
prompt = """

Eres un asistente de ventas virtual llamado Taylor. Estás altamente capacitado para atender a preguntas relacionadas con el inventario de la tienda Zara.
Estás trabajando con un dataframe de pandas en Python. El nombre del dataframe es `df`.
Debes usar las herramientas a continuación para responder la pregunta que se te ha planteado:

python_repl_ast: Un intérprete de Python. Úsalo para ejecutar comandos de Python. La entrada debe ser un comando válido de Python. Cuando uses esta herramienta, a veces la salida se muestra abreviada; asegúrate de que no parezca abreviada antes de usarla en tu respuesta.

Usa el siguiente formato:

Pregunta: la pregunta que debes responder
Pensamiento: siempre debes pensar en qué hacer
Acción: la acción a tomar, debe ser una de [python_repl_ast]
Entrada de Acción: la entrada para la acción
Observación: el resultado de la acción
... (este ciclo de Pensamiento/Acción/Entrada de Acción/Observación puede repetirse N veces)
Pensamiento: ahora sé la respuesta final
Respuesta Final: la respuesta final a la pregunta original

Este es el resultado de `print(df.head())`:
{df}

¡Comienza!
Pregunta: {input}
{agent_scratchpad}

"""

In [ ]:
agente = create_csv_agent(llm,
                          archivo,
                          verbose=True,
                          prompt=prompt,
                          allow_dangerous_code=True
                          )

In [ ]:
agente.run(input('Haga una pregunta: '))

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [ ]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    archivo,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [ ]:
agent.run('¿Cuál es el producto más caro?')

# SQL approach

In [4]:
import pandas as pd

df = pd.read_csv("zara.csv", sep=';')
print(df.shape)
print(df.columns.tolist())

(252, 16)
['Product ID', 'Product Position', 'Promotion', 'Product Category', 'Seasonal', 'Sales Volume', 'brand', 'url', 'sku', 'name', 'description', 'price', 'currency', 'scraped_at', 'terms', 'section']


In [5]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///zara.db")
df.to_sql("zara", engine, index=False)

252

In [6]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM zara WHERE price < 10;")

sqlite
['zara']


"[(132007, 'Aisle', 'No', 'Clothing', 'Yes', 2491, 'Zara', 'https://www.zara.com/us/en/retro-sneakers-p12280220.html', '276378605-515-39', 'RETRO SNEAKERS', 'Sneakers. Upper in a combination of pieces and materials. Lacing with seven pairs of eyelets. Quilted back piece. Chunky sole. Retro style.', 9.99, 'USD', '2024-02-19T09:00:34.990048', 'shoes', 'MAN'), (147998, 'Aisle', 'No', 'Clothing', 'No', 2556, 'Zara', 'https://www.zara.com/us/en/retro-high-top-sneakers-p12180220.html', '277776476-700-39', 'RETRO HIGH TOP SNEAKERS', 'High-top sneakers. Upper in a combination of pieces. Laces with ten pairs of eyelets. Chunky sole.', 9.99, 'USD', '2024-02-19T09:00:36.249339', 'shoes', 'MAN'), (136738, 'Front of Store', 'No', 'Clothing', 'Yes', 542, 'Zara', 'https://www.zara.com/us/en/high-collar-knit-sweater-p01509118.html', '287414911-803-2', 'HIGH COLLAR KNIT SWEATER', 'Sweater with high collar and long sleeves. Front pocket. Side vents at hem. Rib trim.', 7.99, 'USD', '2024-02-19T09:10:33.1

In [8]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [9]:
agent_executor.invoke({"input": "¿Cuál es el precio promedio?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


zara
Invoking: `sql_db_schema` with `{'table_names': 'zara'}`



CREATE TABLE zara (
	"Product ID" BIGINT, 
	"Product Position" TEXT, 
	"Promotion" TEXT, 
	"Product Category" TEXT, 
	"Seasonal" TEXT, 
	"Sales Volume" BIGINT, 
	brand TEXT, 
	url TEXT, 
	sku TEXT, 
	name TEXT, 
	description TEXT, 
	price FLOAT, 
	currency TEXT, 
	scraped_at TEXT, 
	terms TEXT, 
	section TEXT
)

/*
3 rows from zara table:
Product ID	Product Position	Promotion	Product Category	Seasonal	Sales Volume	brand	url	sku	name	description	price	currency	scraped_at	terms	section
185102	Aisle	No	Clothing	No	2823	Zara	https://www.zara.com/us/en/basic-puffer-jacket-p06985450.html	272145190-250-2	BASIC PUFFER JACKET	Puffer jacket made of tear-resistant ripstop fabric. High collar and adjustable long sleeves with ad	19.99	USD	2024-02-19T08:50:05.654618	jackets	MAN
188771	Aisle	No	Clothing	No	654	Zara	https://www.zara.com/us/en/tuxedo-

{'input': '¿Cuál es el precio promedio?',
 'output': 'El precio promedio de los productos en la tabla es de aproximadamente 86.25 USD.'}

In [11]:
agent_executor.invoke({"input": "¿Cuál es el precio de las RETRO HIGH TOP SNEAKERS?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


zara
Invoking: `sql_db_schema` with `{'table_names': 'zara'}`



CREATE TABLE zara (
	"Product ID" BIGINT, 
	"Product Position" TEXT, 
	"Promotion" TEXT, 
	"Product Category" TEXT, 
	"Seasonal" TEXT, 
	"Sales Volume" BIGINT, 
	brand TEXT, 
	url TEXT, 
	sku TEXT, 
	name TEXT, 
	description TEXT, 
	price FLOAT, 
	currency TEXT, 
	scraped_at TEXT, 
	terms TEXT, 
	section TEXT
)

/*
3 rows from zara table:
Product ID	Product Position	Promotion	Product Category	Seasonal	Sales Volume	brand	url	sku	name	description	price	currency	scraped_at	terms	section
185102	Aisle	No	Clothing	No	2823	Zara	https://www.zara.com/us/en/basic-puffer-jacket-p06985450.html	272145190-250-2	BASIC PUFFER JACKET	Puffer jacket made of tear-resistant ripstop fabric. High collar and adjustable long sleeves with ad	19.99	USD	2024-02-19T08:50:05.654618	jackets	MAN
188771	Aisle	No	Clothing	No	654	Zara	https://www.zara.com/us/en/tuxedo-

{'input': '¿Cuál es el precio de las RETRO HIGH TOP SNEAKERS?',
 'output': 'El precio de las RETRO HIGH TOP SNEAKERS es de 9.99 USD.'}

In [12]:
agent_executor.invoke({"input": "¿Cuál es el producto más caro?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


zara
Invoking: `sql_db_schema` with `{'table_names': 'zara'}`



CREATE TABLE zara (
	"Product ID" BIGINT, 
	"Product Position" TEXT, 
	"Promotion" TEXT, 
	"Product Category" TEXT, 
	"Seasonal" TEXT, 
	"Sales Volume" BIGINT, 
	brand TEXT, 
	url TEXT, 
	sku TEXT, 
	name TEXT, 
	description TEXT, 
	price FLOAT, 
	currency TEXT, 
	scraped_at TEXT, 
	terms TEXT, 
	section TEXT
)

/*
3 rows from zara table:
Product ID	Product Position	Promotion	Product Category	Seasonal	Sales Volume	brand	url	sku	name	description	price	currency	scraped_at	terms	section
185102	Aisle	No	Clothing	No	2823	Zara	https://www.zara.com/us/en/basic-puffer-jacket-p06985450.html	272145190-250-2	BASIC PUFFER JACKET	Puffer jacket made of tear-resistant ripstop fabric. High collar and adjustable long sleeves with ad	19.99	USD	2024-02-19T08:50:05.654618	jackets	MAN
188771	Aisle	No	Clothing	No	654	Zara	https://www.zara.com/us/en/tuxedo-

{'input': '¿Cuál es el producto más caro?',
 'output': 'El producto más caro es la "CROPPED LEATHER JACKET" con un precio de 439.0 USD.'}

In [13]:
agent_executor.invoke({"input": "¿Cuál es el producto más barato?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


zara
Invoking: `sql_db_schema` with `{'table_names': 'zara'}`



CREATE TABLE zara (
	"Product ID" BIGINT, 
	"Product Position" TEXT, 
	"Promotion" TEXT, 
	"Product Category" TEXT, 
	"Seasonal" TEXT, 
	"Sales Volume" BIGINT, 
	brand TEXT, 
	url TEXT, 
	sku TEXT, 
	name TEXT, 
	description TEXT, 
	price FLOAT, 
	currency TEXT, 
	scraped_at TEXT, 
	terms TEXT, 
	section TEXT
)

/*
3 rows from zara table:
Product ID	Product Position	Promotion	Product Category	Seasonal	Sales Volume	brand	url	sku	name	description	price	currency	scraped_at	terms	section
185102	Aisle	No	Clothing	No	2823	Zara	https://www.zara.com/us/en/basic-puffer-jacket-p06985450.html	272145190-250-2	BASIC PUFFER JACKET	Puffer jacket made of tear-resistant ripstop fabric. High collar and adjustable long sleeves with ad	19.99	USD	2024-02-19T08:50:05.654618	jackets	MAN
188771	Aisle	No	Clothing	No	654	Zara	https://www.zara.com/us/en/tuxedo-

{'input': '¿Cuál es el producto más barato?',
 'output': 'El producto más barato es el "HIGH COLLAR KNIT SWEATER" con un precio de 7.99 USD.'}